In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import fiona
import pyproj
import contextily
import plotly.express as px
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
path = "/srv/data/my_shared_data_folder/cpr/"

dir_list = os.listdir(path)
#dir_in = os.listdir(os.getcwd())
#directory = pd.Series(dir_in)
#print(dir_in)
print(dir_list)

['california_counties_census_data.csv', 'cleaned_county_data.csv', 'R13391251_SL140.csv', 'R13391250_SL050.csv', 'california_tracts.gpkg', 'pur2020_nonag.parquet', 'cleaned_tract_data.csv', 'cleaned_elsd_data.csv', 'california_zctas.gpkg', 'california_counties.gpkg', 'vision_providers_geocoded.csv', 'california_watersheds.gpkg', 'california_zcta_census_data.csv', 'R13391252_SL950.csv', 'new_test_file.gpkg', 'readme.md', 'california_elementary_school_districts.gpkg', 'pur2020_ag.parquet', 'california_tracts_census_data.csv', 'cleaned_zcta_data.csv', 'california_elementary_schools_census_data.csv', 'R13391293_SL860.csv', 'another_copy_pur2020_ag.parquet']


In [2]:
def Cfp_to_CfpString(num):
    Cfp = str(num)
    if (len(Cfp) == 1):
        fix = '00' + Cfp
    elif (len(Cfp) == 2):
        fix = '0' + Cfp
    elif (len(Cfp) == 3):
        fix = Cfp
    else: return np.nan

    return fix

def tract_fixer(num):
    num = str(num)
    if (len(num) >= 2) & (len(num) <=6):
        zero_times = 6 - len(num)
        fix = zero_times*'0' + num
        return fix
    else: return np.nan

In [3]:
features = pd.read_csv('../data/pesticide_5_w_tracts.csv', low_memory = False) 
features.drop(columns = ['Unnamed: 0'], inplace = True)

In [4]:
features['COUNTYFP'] = features['COUNTYFP'].apply(Cfp_to_CfpString)

In [5]:
features['TRACTFP'] = features['TRACTCE'].apply(tract_fixer)

In [6]:
features['PU_FIPS'] = '06' + features['COUNTYFP'] + features['TRACTFP']

In [7]:
features['MTownshipRange'] = features.Meridian + features.TownshipRange

In [8]:
features['test'] = 1

In [9]:
features.head(2)

,YEAR,COUNTY_NAME,COMTRS,POUNDS_PRODUCT_APPLIED,AMOUNT_TREATED,AERIAL_GROUND_INDICATOR,Meridian,County_Code,Section,air,carcinogens,endo_disrupt,ch_I,fumigants,neonicotinoids,repro_dev,TownshipRange,COUNTYFP,NAME_fromtracts,TRACTCE,OBJECTID,TRACTFP,PU_FIPS,MTownshipRange,test
0,2018,FRESNO,10M11S13E15,486.7475,69.8,G,MDM,10,15.0,True,False,False,False,False,False,False,T11S R13E,019,84.02,8402,3693,008402,06019008402,MDMT11S R13E,1
1,2018,FRESNO,10M11S13E24,0.2087,63.0,G,MDM,10,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T11S R13E,019,84.02,8402,3693,008402,06019008402,MDMT11S R13E,1


In [10]:
# open to plot tract level
places = gpd.read_file('../data/tract_w_geo.geojson')
f = open('../data/tract_w_geo.geojson')
places1 = json.load(f)

In [10]:
# open to plot TownshipRange level
places = gpd.read_file('./places_w_MTownship.geojson')
g = open('./places_w_MTownship.geojson')
places2 = json.load(g)

In [9]:
# sorted(list(places['FIPS GEOID'].unique()))

In [10]:
places.head(2)

,STATEFP,COUNTYFP,TRACTFP,FIPS GEOID,NAME,Geo_QNAME,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON,FIPS,state,tract,geoid,area_name,area_land,total_pop,housing_units,owner_occupied_housing_units,persons_no_health_insurance,families,families_income_below_poverty_level,median_household_income,nh_white_alone,nh_black_or_african_american_alone,nh_american_indian_and_alaska_native_alone,nh_asian_alone,nh_native_hawaiian_and_other_pacific_islander_alone,nh_other,nh_two_or_more,hispanic_or_latino,households_receiving_assistance,households,pct_owner_occupied,pct_no_health_insurance,pct_families_income_below_poverty_level,pct_households_receiving_assistance,pct_nh_white_alone,pct_nh_black_or_african_american_alone,pct_nh_american_indian_and_alaska_native_alone,pct_nh_asian_alone,pct_nh_native_hawaiian_and_other_pacific_islander_alone,pct_nh_other,pct_nh_two_or_more,pct_hispanic_or_latino,geometry
0,06,079,010707,06079010707,107.07,Census Tract 107.07,G5020,14046662,10955892,+35.3320895,-120.8635035,06079010707,ca,10707.0,14000US06079010707,Census Tract 107.07,5.423278,6385.0,2957.0,2150.0,409.0,1677.0,91.0,98125.0,4867.0,0.0,34.0,199.0,0.0,0.0,198.0,1087.0,946.0,2750.0,0.727088,0.064056,0.054264,0.344000,0.762255,0.000000,0.005325,0.031167,0.0,0.0,0.031010,0.170243,"MULTIPOLYGON (((-120.85931 35.31747, -120.8591..."
1,06,079,010703,06079010703,107.03,Census Tract 107.03,G5020,4008527,0,+35.3185488,-120.8212247,06079010703,ca,10703.0,14000US06079010703,Census Tract 107.03,1.547702,3825.0,1436.0,656.0,254.0,951.0,104.0,62333.0,2401.0,34.0,5.0,196.0,0.0,0.0,93.0,1096.0,932.0,1386.0,0.456825,0.066405,0.109359,0.672439,0.627712,0.008889,0.001307,0.051242,0.0,0.0,0.024314,0.286536,"MULTIPOLYGON (((-120.83326 35.32008, -120.8332..."


In [5]:
#set(features_F.TRACTCE).difference(set(places_F.tract))

In [11]:
def plotr2(features, column, plot = 'TownshipRange', year = 2018,pesticide_type = '', county = []):
    # 1. filtering for county
    if (county != []):
        features = features.loc[features['COUNTY_NAME'].isin(county)]
    
    # 2. filtering for year
    features = features.loc[features['YEAR'] == year]
    
    # 3. filtering for pesticide type
    if (pesticide_type != ''):
        if (pesticide_type in (list(features.columns))):
            features = features.loc[features[f'{pesticide_type}'] == True]
        else: print('invalid pesticide type')
    # 4. merging dataframe
    if (plot == 'tracts'):
        features = features.groupby(by = 'PU_FIPS').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places1,
                           featureidkey='properties.FIPS GEOID',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           # range_color=(0,1_900_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
   
    elif (plot == 'TownshipRange'):
        features = features.groupby(by = 'MTownshipRange').sum()[[column]]
        fig = px.choropleth_mapbox(features, 
                           geojson=places2,
                           featureidkey='properties.MTownshipRange',
                           locations=features.index.to_series(), 
                           color=column,
                           color_continuous_scale="Viridis",
                           #range_color=(0,2_000),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat":36.746841 , "lon": -119.772591},
                           opacity=0.4,
                           #labels={'chem_type':'chemical type'}
                          )
    else: return 'unable to plot'
        
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        
    return fig

In [16]:
figure = plotr2(features,column = 'POUNDS_PRODUCT_APPLIED',year = 2018 ,pesticide_type = '' ,county = ['FRESNO'], plot = 'tracts')

In [1]:
#figure

In [12]:
figure2 = plotr2(features,column = 'POUNDS_PRODUCT_APPLIED',year = 2018 ,pesticide_type = '' ,county = ['FRESNO'])

In [14]:
#figure2

In [ ]:
#set(features.TRACTCE).difference(set(placesF.tract))

In [14]:
#len(features.loc[features['COUNTY_NAME'] == 'FRESNO'].OBJECTID.unique())

95